In [1]:
#!pip install mne scipy
#!pip install pandas numpy openpyxl
#!pip install tsfresh
#!pip install PyWavelets

In [107]:
import os
import numpy as np
import scipy.signal as signal
import mne

def process_all_eeg_data() -> dict:
    """
    Process all .bdf EEG files in the current directory, applying filters and extracting data from
    channels A15 (O1), A16 (Oz), and A17 (O2).

    Returns
    -------
    dict
        A dictionary containing processed EEG data and header information for each file.
    """
    # Get a list of all .bdf files in the current directory
    files = [f for f in os.listdir('.') if f.endswith('.bdf')]
    if not files:
        raise FileNotFoundError("No BDF files found in the current directory")
    
    # Initialize the results dictionary
    results = {}
    
    # Loop over each file
    for filename in files:
        full_file_path = os.path.join(os.getcwd(), filename)
        
        # Read the raw EEG data using MNE
        raw = mne.io.read_raw_bdf(full_file_path, preload=True)
        hdr = raw.info
        
        # Select data from channels A15 (O1), A16 (Oz), and A17 (O2)
        channels_select = ['A15', 'A16', 'A17']
        missing_channels = [ch for ch in channels_select if ch not in hdr['ch_names']]
        if missing_channels:
            raise ValueError(f"Selected channels {missing_channels} not found in the data")
        
        channel_indices = [hdr['ch_names'].index(ch) for ch in channels_select]
        EEG_data = raw.get_data(picks=channel_indices).T  # Shape: (n_samples, n_channels)
        
        # Filter EEG Data
        Fs = hdr['sfreq']  # Sampling frequency
        
        # Bandpass filter parameters (2 to 80 Hz)
        Fc_BP = [2, 80]  # Bandpass frequency range
        Wn_BP = [f / (Fs / 2) for f in Fc_BP]  # Normalize by Nyquist frequency
        
        # Create and apply bandpass filter (6th order zero-phase Butterworth IIR)
        B_BP, A_BP = signal.butter(3, Wn_BP, btype='bandpass')
        EEG_filtered_BP = signal.filtfilt(B_BP, A_BP, EEG_data, axis=0)
        
        # Band stop filter parameters (48 to 52 Hz)
        Fc_BS = [48, 52]  # Band stop frequency range
        Wn_BS = [f / (Fs / 2) for f in Fc_BS]  # Normalize by Nyquist frequency
        
        # Create and apply band stop filter (6th order zero-phase Butterworth IIR)
        B_BS, A_BS = signal.butter(3, Wn_BS, btype='bandstop')
        EEG_filtered = signal.filtfilt(B_BS, A_BS, EEG_filtered_BP, axis=0)
        
        # Extract prefix before underscore from the filename
        underscore_index = filename.find('_')
        if underscore_index == -1:
            raise ValueError(f"Filename format error, no underscore found in {filename}")
        key = filename[:underscore_index]
        
        # Store results in the dictionary
        results[key] = {
            'data': EEG_filtered,      # Filtered data for channels A15, A16, A17
            'channels': channels_select,  # List of channel names
            'header': hdr
        }
        
        # Display a message indicating successful processing
        print(f"Data for file {filename} processed successfully")
    
    return results


In [108]:
results = process_all_eeg_data()

Extracting EDF parameters from c:\Users\WERPELGA\OneDrive - Danone\Desktop\UoA\2024.1&2\Python Gabe\A1_Full_Block.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 739327  =      0.000 ...   361.000 secs...
Data for file A1_Full_Block.bdf processed successfully
Extracting EDF parameters from c:\Users\WERPELGA\OneDrive - Danone\Desktop\UoA\2024.1&2\Python Gabe\A3_Full_Block.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 757759  =      0.000 ...   370.000 secs...
Data for file A3_Full_Block.bdf processed successfully
Extracting EDF parameters from c:\Users\WERPELGA\OneDrive - Danone\Desktop\UoA\2024.1&2\Python Gabe\A4_Full_Block.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 782335  =      0.000 ...   382.000 secs...
Data for file A4_Full_Block.bdf processed successfully
Extracting EDF parameters from c:\Users\WERPELGA\One

In [125]:
import numpy as np
import pandas as pd

def segment_eeg_data_new(results: dict, cohort_file: str = 'Cohort.xlsx') -> dict:
    """
    Segments EEG data into predefined sections (EC, EO, LC, RC, DEC, NDEC) based on cohort information,
    removing the first 2 seconds from each section.

    Parameters
    ----------
    results : dict
        Dictionary containing the raw EEG data and header information for each key (participant).
    cohort_file : str, optional
        Path to the Excel file containing cohort information (default is 'Cohort.xlsx').

    Returns
    -------
    dict
        Dictionary containing segmented EEG data for each participant.
    """
    # Read the cohort information from an Excel file
    cohort_table = pd.read_excel(cohort_file)
    # Segment Duration (in seconds)
    segment_duration = 10  # Original segment duration in seconds
    skip_duration = 2      # Duration to skip at the start of each segment (2 seconds)

    # Initialize the segmented results dictionary
    segmented_data = {}

    # Iterate through each key in the results dictionary
    for key, result in results.items():
        data = result['data']  # Data shape: (n_samples, n_channels)
        hdr = result['header']

        # Find the matching row in the cohort table
        cohort_row = cohort_table[cohort_table['Cohort'] == key]
        
        if cohort_row.empty:
            raise ValueError(f"Cohort information not found for {key}")

        # Define the sample rate and calculate sample counts
        Fs = hdr['sfreq']  # Sampling frequency
        samples_per_segment = int(segment_duration * Fs)
        samples_to_skip = int(skip_duration * Fs)
        effective_samples_per_segment = samples_per_segment - samples_to_skip
        n_channels = data.shape[1]  # Number of channels (should be 3: O1, Oz, O2)

        # Initialize segments with zeros
        EC = np.zeros((effective_samples_per_segment, n_channels))
        EO = np.zeros((effective_samples_per_segment, n_channels))
        LC = np.zeros((effective_samples_per_segment, n_channels))
        RC = np.zeros((effective_samples_per_segment, n_channels))
        DEC = np.zeros((effective_samples_per_segment, n_channels))
        NDEC = np.zeros((effective_samples_per_segment, n_channels))

        # Fill segments with data if available, skipping the first 2 seconds
        # EC segment
        segment_start = 0
        segment_end = samples_per_segment
        if data.shape[0] >= segment_end:
            EC = data[segment_start + samples_to_skip : segment_end, :]
        else:
            print(f"Not enough data for EC segment in {key}")

        # EO segment
        segment_start = samples_per_segment
        segment_end = 2 * samples_per_segment
        if data.shape[0] >= segment_end:
            EO = data[segment_start + samples_to_skip : segment_end, :]
        else:
            print(f"Not enough data for EO segment in {key}")

        # LC segment
        segment_start = 2 * samples_per_segment
        segment_end = 3 * samples_per_segment
        if data.shape[0] >= segment_end:
            LC = data[segment_start + samples_to_skip : segment_end, :]
        else:
            print(f"Not enough data for LC segment in {key}")

        # RC segment
        segment_start = 3 * samples_per_segment
        segment_end = 4 * samples_per_segment
        if data.shape[0] >= segment_end:
            RC = data[segment_start + samples_to_skip : segment_end, :]
        else:
            print(f"Not enough data for RC segment in {key}")

        # Apply conditions based on cohort table
        if cohort_row['LC'].values[0] == 'DEC':
            # Assign 'DEC' to LC and 'NDEC' to RC
            DEC = LC
            NDEC = RC
        elif cohort_row['RC'].values[0] == 'DEC':
            # Assign 'DEC' to RC and 'NDEC' to LC
            DEC = RC
            NDEC = LC
        else:
            # If neither LC nor RC is 'DEC', assign NDEC accordingly
            NDEC = LC
            # Optionally handle cases where DEC is not specified
            DEC = RC  # Or set DEC to zeros if appropriate

        # Store the segmented data and 'LinesDifference' in the results dictionary
        segmented_data[key] = {
            'header': hdr,
            'EC': EC,
            'EO': EO,
            'DEC': DEC,
            'NDEC': NDEC,
            'LinesDifference': cohort_row['LinesDifference'].values[0]
        }

    return segmented_data


In [126]:
segmented_data = segment_eeg_data_new(results)

In [127]:
import pandas as pd
import numpy as np
from tsfresh import extract_features, select_features
from tsfresh.utilities.dataframe_functions import impute
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

def prepare_time_series_by_section(segmented_data, cohort_table):
    """
    Prepares a DataFrame suitable for tsfresh from segmented EEG data for all sections (EC, EO, DEC, NDEC).

    Parameters
    ----------
    segmented_data : dict
        The dictionary containing segmented EEG data for each participant.
    cohort_table : pd.DataFrame
        DataFrame containing cohort information (including labels for Amblyopia/Control).

    Returns
    -------
    pd.DataFrame, pd.Series
        A DataFrame where each row represents a time-series sample with columns 'id', 'time', 'O1', 'Oz', 'O2',
        and a Series with group labels indexed by 'id'.
    """
    data_list = []
    labels_list = []

    # Loop through each participant's data
    for key, value in segmented_data.items():
        # Find the matching cohort row
        cohort_row = cohort_table[cohort_table['Cohort'] == key]
        if cohort_row.empty:
            continue

        # Assign label based on the first letter of the 'Cohort' column (Amblyopia = 1, Control = 0)
        label = 1 if key.startswith('A') else 0

        # Get channel names; default to ['O1', 'Oz', 'O2'] if not available
        channels = value.get('channels', ['O1', 'Oz', 'O2'])

        # For each section (EC, EO, DEC, NDEC)
        for section in ['EC', 'EO', 'DEC', 'NDEC']:
            section_data = value[section]  # Shape: (n_samples, n_channels)

            # Create a DataFrame for this section
            n_samples = section_data.shape[0]
            df = pd.DataFrame({
                'id': f"{key}_{section}",
                'time': np.arange(n_samples)
            })

            # Add each channel's data as a column
            for idx, channel_name in enumerate(channels):
                df[channel_name] = section_data[:, idx]

            # Append to data list
            data_list.append(df)

            # Append label for this 'id' (participant_section)
            labels_list.append({'id': f"{key}_{section}", 'label': label})

    # Concatenate all data into a single DataFrame
    time_series_df = pd.concat(data_list, ignore_index=True)

    # Create a labels DataFrame and convert to a Series indexed by 'id'
    labels_df = pd.DataFrame(labels_list).drop_duplicates(subset='id')
    labels_series = labels_df.set_index('id')['label']

    # Return the time-series data and corresponding labels
    return time_series_df, labels_series

# Load your cohort table (must include 'Cohort' column)
cohort_table = pd.read_excel('Cohort.xlsx')

# Prepare the time series DataFrame and labels
time_series_df, labels = prepare_time_series_by_section(segmented_data, cohort_table)


In [128]:
# Assuming your dataframe is named 'df'
time_series_df_0 = time_series_df[(time_series_df['time'] >= 0) & (time_series_df['time'] <= 1637)]
time_series_df_1 = time_series_df[(time_series_df['time'] >= 1638) & (time_series_df['time'] <= 3276)]
time_series_df_2 = time_series_df[(time_series_df['time'] >= 3276) & (time_series_df['time'] <= 4914)]
time_series_df_3 = time_series_df[(time_series_df['time'] >= 4914) & (time_series_df['time'] <= 6552)]
time_series_df_4 = time_series_df[(time_series_df['time'] >= 6552) & (time_series_df['time'] <= 8190)]
time_series_df_5 = time_series_df[(time_series_df['time'] >= 8190) & (time_series_df['time'] <= 9828)]
time_series_df_6 = time_series_df[(time_series_df['time'] >= 9828) & (time_series_df['time'] <= 11466)]
time_series_df_7 = time_series_df[(time_series_df['time'] >= 11466) & (time_series_df['time'] <= 13104)]
time_series_df_8 = time_series_df[(time_series_df['time'] >= 13104) & (time_series_df['time'] <= 14742)]
time_series_df_9 = time_series_df[(time_series_df['time'] >= 14742) & (time_series_df['time'] <= 16380)]

In [149]:
# Import necessary libraries
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np
import gc
from tsfresh import extract_features
from tsfresh.feature_extraction import ComprehensiveFCParameters, EfficientFCParameters
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_extraction import settings
import pickle

# Define the function to process data in chunks using ComprehensiveFCParameters
def process_in_chunks(time_series_df, N):
    # Get unique IDs
    unique_ids = time_series_df['id'].unique()
    
    # Split the unique IDs into chunks of size N
    chunks = [unique_ids[i:i + N] for i in range(0, len(unique_ids), N)]
    
    # Initialize an empty list to store the results
    results = []
    
    # Process each chunk
    for chunk in chunks:
        # Filter the DataFrame to include only the IDs in the current chunk
        chunk_df = time_series_df[time_series_df['id'].isin(chunk)]
        
        # Extract features for the current chunk using ComprehensiveFCParameters
        extracted_features_chunk = extract_features(
            chunk_df,
            column_id='id',
            column_sort='time',
            default_fc_parameters=ComprehensiveFCParameters(),  # Use ComprehensiveFCParameters() for more features
            n_jobs=4,  # Adjust based on your CPU cores
            # Since data is in wide format, we do not need to specify column_kind and column_value
        )
        
        # Impute missing values in the extracted features
        impute(extracted_features_chunk)
        
        # Append the extracted features to the results list
        results.append(extracted_features_chunk)
        
        # Clear memory
        del chunk_df, extracted_features_chunk
        gc.collect()
    
    # Concatenate all the results into a single DataFrame
    final_result = pd.concat(results)
    
    return final_result


In [55]:
# 1. Put all DataFrames into a List
time_series_dfs = [
    time_series_df_0,
    time_series_df_1,
    time_series_df_2,
    time_series_df_3,
    time_series_df_4,
    time_series_df_5,
    time_series_df_6,
    time_series_df_7,
    time_series_df_8,
    time_series_df_9
]

# Set the chunk size N (adjust based on your memory constraints)
N = 10  # Smaller chunk size to manage memory usage

# Initialize a set to store all selected feature names
all_selected_feature_names = set()

# Loop over each DataFrame and extract features
for idx, ts_df in enumerate(time_series_dfs):
    print(f"Processing DataFrame {idx}...")
    
    # 2. Extract features using the process_in_chunks function
    extracted_features = process_in_chunks(ts_df, N)
    
    # Clean the extracted features
    extracted_features_clean = extracted_features.replace([np.inf, -np.inf], np.nan).dropna(axis=1)
    
    # Ensure that the labels are aligned with the extracted features
    # Assuming 'labels' is a Series with 'id' as the index
    labels_aligned = labels.loc[extracted_features_clean.index]
    
    # Check if we have enough data to split
    if len(extracted_features_clean) < 2:
        print(f"Not enough data in DataFrame {idx} for training and testing. Skipping...")
        continue
    
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(
        extracted_features_clean,
        labels_aligned,
        test_size=0.2,
        random_state=42,
        stratify=labels_aligned  # Ensure stratified sampling
    )
    
    # Perform feature selection
    selector = SelectKBest(f_classif, k=10)  # Adjust 'k' as needed
    X_train_selected = selector.fit_transform(X_train, y_train)
    
    # Get the names of the selected features
    selected_feature_names = extracted_features_clean.columns[selector.get_support()]
    
    # Add the selected features to the set
    all_selected_feature_names.update(selected_feature_names)
    
    # Optionally, you can proceed to train models on the selected features here
    # For brevity, this step is omitted

# 3. Combine All Selected Features
print(f"Total unique selected features from all DataFrames: {len(all_selected_feature_names)}")
print(all_selected_feature_names)


Processing DataFrame 0...


Feature Extraction: 100%|██████████| 15/15 [00:30<00:00,  2.01s/it]
c:\Users\WERPELGA\AppData\Local\Programs\Python\Python39\lib\site-packages\tsfresh\utilities\dataframe_functions.py:198: RuntimeWarning: The columns ['Oz__autocorrelation__lag_0' 'Oz__autocorrelation__lag_1'
 'Oz__autocorrelation__lag_2' 'Oz__autocorrelation__lag_3'
 'Oz__autocorrelation__lag_4' 'Oz__autocorrelation__lag_5'
 'Oz__autocorrelation__lag_6' 'Oz__autocorrelation__lag_7'
 'Oz__autocorrelation__lag_8' 'Oz__autocorrelation__lag_9'
 'Oz__query_similarity_count__query_None__threshold_0.0'
 'O2__autocorrelation__lag_0' 'O2__autocorrelation__lag_1'
 'O2__autocorrelation__lag_2' 'O2__autocorrelation__lag_3'
 'O2__autocorrelation__lag_4' 'O2__autocorrelation__lag_5'
 'O2__autocorrelation__lag_6' 'O2__autocorrelation__lag_7'
 'O2__autocorrelation__lag_8' 'O2__autocorrelation__lag_9'
 'O2__query_similarity_count__query_None__threshold_0.0'
 'O1__autocorrelation__lag_0' 'O1__autocorrelation__lag_1'
 'O1__autocorrelatio

Processing DataFrame 1...


Feature Extraction: 100%|██████████| 15/15 [00:26<00:00,  1.80s/it]
c:\Users\WERPELGA\AppData\Local\Programs\Python\Python39\lib\site-packages\tsfresh\utilities\dataframe_functions.py:198: RuntimeWarning: The columns ['Oz__autocorrelation__lag_0' 'Oz__autocorrelation__lag_1'
 'Oz__autocorrelation__lag_2' 'Oz__autocorrelation__lag_3'
 'Oz__autocorrelation__lag_4' 'Oz__autocorrelation__lag_5'
 'Oz__autocorrelation__lag_6' 'Oz__autocorrelation__lag_7'
 'Oz__autocorrelation__lag_8' 'Oz__autocorrelation__lag_9'
 'Oz__query_similarity_count__query_None__threshold_0.0'
 'O2__autocorrelation__lag_0' 'O2__autocorrelation__lag_1'
 'O2__autocorrelation__lag_2' 'O2__autocorrelation__lag_3'
 'O2__autocorrelation__lag_4' 'O2__autocorrelation__lag_5'
 'O2__autocorrelation__lag_6' 'O2__autocorrelation__lag_7'
 'O2__autocorrelation__lag_8' 'O2__autocorrelation__lag_9'
 'O2__query_similarity_count__query_None__threshold_0.0'
 'O1__autocorrelation__lag_0' 'O1__autocorrelation__lag_1'
 'O1__autocorrelatio

Processing DataFrame 2...


...

Feature Extraction: 100%|██████████| 15/15 [00:36<00:00,  2.46s/it]
c:\Users\WERPELGA\AppData\Local\Programs\Python\Python39\lib\site-packages\tsfresh\utilities\dataframe_functions.py:198: RuntimeWarning: The columns ['O1__autocorrelation__lag_0' 'O1__autocorrelation__lag_1'
 'O1__autocorrelation__lag_2' 'O1__autocorrelation__lag_3'
 'O1__autocorrelation__lag_4' 'O1__autocorrelation__lag_5'
 'O1__autocorrelation__lag_6' 'O1__autocorrelation__lag_7'
 'O1__autocorrelation__lag_8' 'O1__autocorrelation__lag_9'
 'O1__query_similarity_count__query_None__threshold_0.0'
 'Oz__autocorrelation__lag_0' 'Oz__autocorrelation__lag_1'
 'Oz__autocorrelation__lag_2' 'Oz__autocorrelation__lag_3'
 'Oz__autocorrelation__lag_4' 'Oz__autocorrelation__lag_5'
 'Oz__autocorrelation__lag_6' 'Oz__autocorrelation__lag_7'
 'Oz__autocorrelation__lag_8' 'Oz__autocorrelation__lag_9'
 'Oz__query_similarity_count__query_None__threshold_0.0'
 'O2__autocorrelation__lag_0' 'O2__autocorrelation__lag_1'
 'O2__autocorrelatio

Processing DataFrame 9...


Feature Extraction: 100%|██████████| 15/15 [00:31<00:00,  2.12s/it]
c:\Users\WERPELGA\AppData\Local\Programs\Python\Python39\lib\site-packages\tsfresh\utilities\dataframe_functions.py:198: RuntimeWarning: The columns ['O2__autocorrelation__lag_0' 'O2__autocorrelation__lag_1'
 'O2__autocorrelation__lag_2' 'O2__autocorrelation__lag_3'
 'O2__autocorrelation__lag_4' 'O2__autocorrelation__lag_5'
 'O2__autocorrelation__lag_6' 'O2__autocorrelation__lag_7'
 'O2__autocorrelation__lag_8' 'O2__autocorrelation__lag_9'
 'O2__query_similarity_count__query_None__threshold_0.0'
 'O1__autocorrelation__lag_0' 'O1__autocorrelation__lag_1'
 'O1__autocorrelation__lag_2' 'O1__autocorrelation__lag_3'
 'O1__autocorrelation__lag_4' 'O1__autocorrelation__lag_5'
 'O1__autocorrelation__lag_6' 'O1__autocorrelation__lag_7'
 'O1__autocorrelation__lag_8' 'O1__autocorrelation__lag_9'
 'O1__query_similarity_count__query_None__threshold_0.0'
 'Oz__autocorrelation__lag_0' 'Oz__autocorrelation__lag_1'
 'Oz__autocorrelatio

Total unique selected features from all DataFrames: 50


ValueError: columns cannot be a set

In [63]:
all_selected_feature_names

{'O1__ar_coefficient__coeff_2__k_10',
 'O1__ar_coefficient__coeff_3__k_10',
 'O1__ar_coefficient__coeff_4__k_10',
 'O1__ar_coefficient__coeff_5__k_10',
 'O1__ar_coefficient__coeff_6__k_10',
 'O1__ar_coefficient__coeff_7__k_10',
 'O1__ar_coefficient__coeff_8__k_10',
 'O1__ar_coefficient__coeff_9__k_10',
 'O1__augmented_dickey_fuller__attr_"usedlag"__autolag_"AIC"',
 'O1__fft_coefficient__attr_"angle"__coeff_40',
 'O1__fft_coefficient__attr_"angle"__coeff_41',
 'O1__fft_coefficient__attr_"angle"__coeff_57',
 'O1__fft_coefficient__attr_"angle"__coeff_6',
 'O1__fft_coefficient__attr_"angle"__coeff_63',
 'O1__fft_coefficient__attr_"angle"__coeff_76',
 'O1__fft_coefficient__attr_"angle"__coeff_87',
 'O1__fft_coefficient__attr_"imag"__coeff_40',
 'O1__fft_coefficient__attr_"imag"__coeff_41',
 'O1__fft_coefficient__attr_"imag"__coeff_63',
 'O1__fft_coefficient__attr_"real"__coeff_10',
 'O1__fft_coefficient__attr_"real"__coeff_11',
 'O1__fft_coefficient__attr_"real"__coeff_3',
 'O1__fft_coeffic

In [150]:
time_series_df

,id,time,O1,Oz,O2
0,A1_EC,0,0.000010,-1.500819e-06,-0.000006
1,A1_EC,1,0.000010,-1.018319e-06,-0.000005
2,A1_EC,2,0.000010,-6.288356e-07,-0.000004
3,A1_EC,3,0.000010,-3.342138e-07,-0.000003
4,A1_EC,4,0.000010,-1.304634e-07,-0.000002
...,...,...,...,...,...
851963,C1_NDEC,16379,-0.000003,-3.512712e-06,-0.000004
851964,C1_NDEC,16380,-0.000003,-2.912297e-06,-0.000004
851965,C1_NDEC,16381,-0.000003,-2.468121e-06,-0.000003
851966,C1_NDEC,16382,-0.000003,-2.206402e-06,-0.000003


In [151]:
# Import necessary libraries
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np
import gc
from tsfresh import extract_features
from tsfresh.feature_extraction import EfficientFCParameters, MinimalFCParameters
from tsfresh.utilities.dataframe_functions import impute
from sklearn.metrics import confusion_matrix

# 1. Create a Custom fc_parameters Dictionary
# Load EfficientFCParameters
efficient_fc_parameters = EfficientFCParameters()

# Create custom fc_parameters with the specified feature calculators
custom_fc_parameters = {
    'fft_coefficient': [
        {'coeff': coeff, 'attr': attr}
        for coeff in range(100)  # Adjust the range as needed
        for attr in ['real', 'imag', 'abs', 'angle']
    ],
    'ar_coefficient': [
        {'k': 10, 'coeff': coeff} for coeff in range(1, 11)
    ],
    'augmented_dickey_fuller': [
        {'attr': 'teststat'},
        {'attr': 'pvalue'},
        {'attr': 'usedlag'}
    ],
    'large_standard_deviation': [
        {'r': 0.2}
    ],
    'number_peaks': [
        {'n': n} for n in [1, 5]
    ],
    'fourier_entropy': [
        {'bins': 10}
    ],
    'ratio_beyond_r_sigma': [
        {'r': 1}
    ]
}

# Append MinimalFCParameters to custom_fc_parameters
minimal_fc_parameters = MinimalFCParameters()
custom_fc_parameters.update(minimal_fc_parameters)

# 2. Define the function to process data in chunks using the custom fc_parameters
def process_in_chunks(time_series_df, N, fc_parameters):
    # Get unique IDs
    unique_ids = time_series_df['id'].unique()
    
    # Split the unique IDs into chunks of size N
    chunks = [unique_ids[i:i + N] for i in range(0, len(unique_ids), N)]
    
    # Initialize an empty list to store the results
    results = []
    
    # Process each chunk
    for chunk in chunks:
        # Filter the DataFrame to include only the IDs in the current chunk
        chunk_df = time_series_df[time_series_df['id'].isin(chunk)]
        
        # Extract features for the current chunk using the custom fc_parameters
        extracted_features_chunk = extract_features(
            chunk_df,
            column_id='id',
            column_sort='time',
            default_fc_parameters=fc_parameters,
            n_jobs=4,
        )
        
        # Impute missing values in the extracted features
        impute(extracted_features_chunk)
        
        # Append the extracted features to the results list
        results.append(extracted_features_chunk)
        
        # Clear memory
        del chunk_df, extracted_features_chunk
        gc.collect()
    
    # Concatenate all the results into a single DataFrame
    final_result = pd.concat(results)
    
    return final_result

# 3. Set the chunk size N (adjust based on your memory constraints)
N = 1  # Adjust based on your memory constraints

# 4. Extract features using the process_in_chunks function with custom_fc_parameters
extracted_features = process_in_chunks(time_series_df, N, custom_fc_parameters)

# 5. Clean the extracted features
extracted_features_clean = extracted_features.replace([np.inf, -np.inf], np.nan).dropna(axis=1)

# 6. Ensure that the labels are aligned with the extracted features
labels_aligned = labels.loc[extracted_features_clean.index]

# 7. Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    extracted_features_clean,
    labels_aligned,
    test_size=0.2,
    random_state=42,
    stratify=labels_aligned
)

# 8. Define classifiers and their parameter grids
classifiers = {
    'Random Forest': {
        'pipeline': Pipeline([
            ('selector', SelectKBest(f_classif, k=50)),
            ('classifier', RandomForestClassifier(random_state=42))
        ]),
        'param_grid': {
            'classifier__n_estimators': [100, 200, 500],
            'classifier__max_depth': [None, 10, 20, 30],
            'classifier__min_samples_split': [2, 5, 10],
        }
    },
    'Gradient Boosting': {
        'pipeline': Pipeline([
            ('selector', SelectKBest(f_classif, k=50)),
            ('classifier', GradientBoostingClassifier(random_state=42))
        ]),
        'param_grid': {
            'classifier__n_estimators': [100, 200],
            'classifier__learning_rate': [0.01, 0.1],
            'classifier__max_depth': [3, 5],
            'classifier__min_samples_split': [2, 5],
        }
    },
    'Neural Network': {
        'pipeline': Pipeline([
            ('scaler', StandardScaler()),
            ('selector', SelectKBest(f_classif, k=50)),
            ('classifier', MLPClassifier(random_state=42, max_iter=1000))
        ]),
        'param_grid': {
            'classifier__hidden_layer_sizes': [(50,), (100,), (50, 50)],
            'classifier__activation': ['tanh', 'relu'],
            'classifier__solver': ['adam', 'sgd'],
            'classifier__alpha': [0.0001, 0.001],
            'classifier__learning_rate': ['constant', 'adaptive'],
        }
    },
    'Support Vector Machine': {
        'pipeline': Pipeline([
            ('scaler', StandardScaler()),
            ('selector', SelectKBest(f_classif, k=50)),
            ('classifier', SVC(random_state=42))
        ]),
        'param_grid': [
            {
                'classifier__kernel': ['linear'],
                'classifier__C': [0.1, 1, 10, 100]
            },
            {
                'classifier__kernel': ['rbf'],
                'classifier__C': [0.1, 1, 10, 100],
                'classifier__gamma': ['scale', 'auto']
            },
            {
                'classifier__kernel': ['poly'],
                'classifier__C': [0.1, 1, 10],
                'classifier__degree': [2, 3],
                'classifier__gamma': ['scale', 'auto']
            }
        ]
    },
    'XGBoost': {
        'pipeline': Pipeline([
            ('selector', SelectKBest(f_classif, k=50)),
            ('classifier', XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss'))
        ]),
        'param_grid': {
            'classifier__n_estimators': [50, 100, 200],
            'classifier__max_depth': [3, 5, 7],
            'classifier__learning_rate': [0.01, 0.1, 0.2],
            'classifier__subsample': [0.5, 0.7, 1.0],
            'classifier__colsample_bytree': [0.5, 0.7, 1.0],
        }
    }
}

# 9. Loop through each classifier, perform grid search, and evaluate
for name, classifier_info in classifiers.items():
    print(f"\nTraining and evaluating {name}...")
    pipeline = classifier_info['pipeline']
    param_grid = classifier_info['param_grid']
    
    grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
    grid_search.fit(X_train, y_train)
    best_pipeline = grid_search.best_estimator_
    y_pred = best_pipeline.predict(X_test)
    
    # Print the best parameters found by GridSearchCV
    print(f"Best parameters for {name}: {grid_search.best_params_}")
    
    # Evaluate the model
    print(f"Classification Report for {name}:")
    print(classification_report(y_test, y_pred))
    
    cm = confusion_matrix(y_test, y_pred)
    print(f"Confusion Matrix for {name}:")
    print(cm)
    
    # For models that provide feature importances, display them
    classifier_step = best_pipeline.named_steps['classifier']
    selector_step = best_pipeline.named_steps['selector']
    selected_feature_indices = selector_step.get_support(indices=True)
    selected_feature_names = extracted_features_clean.columns[selected_feature_indices]
    
    if hasattr(classifier_step, 'feature_importances_'):
        important_features = pd.DataFrame({
            'Feature': selected_feature_names,
            'Importance': classifier_step.feature_importances_
        }).sort_values(by='Importance', ascending=False)
        print(f"Important features for {name}:")
        print(important_features)
    elif hasattr(classifier_step, 'coef_'):
        # For linear models like SVM with linear kernel
        if hasattr(classifier_step, 'kernel') and classifier_step.kernel == 'linear':
            importance = np.abs(classifier_step.coef_[0])
            important_features = pd.DataFrame({
                'Feature': selected_feature_names,
                'Importance': importance
            }).sort_values(by='Importance', ascending=False)
            print(f"Important features for {name}:")
            print(important_features)
        elif isinstance(classifier_step, MLPClassifier):
            # For Neural Network, feature importances are not directly available
            print(f"{name} does not provide feature importances directly.")
        else:
            print(f"{name} does not provide feature importances directly.")
    else:
        print(f"{name} does not provide feature importances directly.")


Feature Extraction: 100%|██████████| 3/3 [00:04<00:00,  1.36s/it]



Training and evaluating Random Forest...


c:\Users\WERPELGA\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [   1  413  421  429  841  849  857 1269 1277] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
c:\Users\WERPELGA\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


Best parameters for Random Forest: {'classifier__max_depth': None, 'classifier__min_samples_split': 2, 'classifier__n_estimators': 200}
Classification Report for Random Forest:
              precision    recall  f1-score   support

           0       0.80      0.80      0.80         5
           1       0.83      0.83      0.83         6

    accuracy                           0.82        11
   macro avg       0.82      0.82      0.82        11
weighted avg       0.82      0.82      0.82        11

Confusion Matrix for Random Forest:
[[4 1]
 [1 5]]
Important features for Random Forest:
                                        Feature  Importance
11    O1__fft_coefficient__attr_"abs"__coeff_26    0.052094
40    O2__fft_coefficient__attr_"abs"__coeff_40    0.042027
33            O1__ar_coefficient__coeff_9__k_10    0.039375
48  O2__fft_coefficient__attr_"angle"__coeff_66    0.039189
38    O2__fft_coefficient__attr_"abs"__coeff_31    0.034614
31            O1__ar_coefficient__coeff_7__k_10

c:\Users\WERPELGA\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [   1  413  421  429  841  849  857 1269 1277] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
c:\Users\WERPELGA\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


...

c:\Users\WERPELGA\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [   1  413  421  429  841  849  857 1269 1277] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
c:\Users\WERPELGA\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


Best parameters for Neural Network: {'classifier__activation': 'relu', 'classifier__alpha': 0.0001, 'classifier__hidden_layer_sizes': (50,), 'classifier__learning_rate': 'constant', 'classifier__solver': 'adam'}
Classification Report for Neural Network:
              precision    recall  f1-score   support

           0       1.00      0.80      0.89         5
           1       0.86      1.00      0.92         6

    accuracy                           0.91        11
   macro avg       0.93      0.90      0.91        11
weighted avg       0.92      0.91      0.91        11

Confusion Matrix for Neural Network:
[[4 1]
 [0 6]]
Neural Network does not provide feature importances directly.

Training and evaluating Support Vector Machine...


c:\Users\WERPELGA\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [   1  413  421  429  841  849  857 1269 1277] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
c:\Users\WERPELGA\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


Best parameters for Support Vector Machine: {'classifier__C': 10, 'classifier__degree': 3, 'classifier__gamma': 'scale', 'classifier__kernel': 'poly'}
Classification Report for Support Vector Machine:
              precision    recall  f1-score   support

           0       1.00      0.80      0.89         5
           1       0.86      1.00      0.92         6

    accuracy                           0.91        11
   macro avg       0.93      0.90      0.91        11
weighted avg       0.92      0.91      0.91        11

Confusion Matrix for Support Vector Machine:
[[4 1]
 [0 6]]
Support Vector Machine does not provide feature importances directly.

Training and evaluating XGBoost...
Best parameters for XGBoost: {'classifier__colsample_bytree': 0.7, 'classifier__learning_rate': 0.01, 'classifier__max_depth': 3, 'classifier__n_estimators': 100, 'classifier__subsample': 0.5}
Classification Report for XGBoost:
              precision    recall  f1-score   support

           0       0.71

c:\Users\WERPELGA\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [   1  413  421  429  841  849  857 1269 1277] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
c:\Users\WERPELGA\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw
c:\Users\WERPELGA\AppData\Local\Programs\Python\Python39\lib\site-packages\xgboost\core.py:158: UserWarning: [10:14:59] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [157]:
extracted_features

,"Oz__fft_coefficient__attr_""real""__coeff_0","Oz__fft_coefficient__attr_""imag""__coeff_0","Oz__fft_coefficient__attr_""abs""__coeff_0","Oz__fft_coefficient__attr_""angle""__coeff_0","Oz__fft_coefficient__attr_""real""__coeff_1","Oz__fft_coefficient__attr_""imag""__coeff_1","Oz__fft_coefficient__attr_""abs""__coeff_1","Oz__fft_coefficient__attr_""angle""__coeff_1","Oz__fft_coefficient__attr_""real""__coeff_2","Oz__fft_coefficient__attr_""imag""__coeff_2",...,O2__sum_values,O2__median,O2__mean,O2__length,O2__standard_deviation,O2__variance,O2__root_mean_square,O2__maximum,O2__absolute_maximum,O2__minimum
A1_EC,-0.001221,0.0,0.001221,180.0,-0.001224,2.899419e-05,0.001224,178.643054,-0.001233,5.820108e-05,...,-0.001270,-8.282399e-07,-7.752495e-08,16384.0,0.000022,5.036721e-10,0.000022,0.000086,0.000088,-0.000088
A1_EO,0.000595,0.0,0.000595,0.0,0.000597,-5.627240e-05,0.000600,-5.380901,0.000605,-1.148255e-04,...,0.000349,-5.934965e-07,2.127831e-08,16384.0,0.000011,1.252477e-10,0.000011,0.000042,0.000044,-0.000044
A1_DEC,-0.000501,0.0,0.000501,180.0,-0.000503,1.029029e-04,0.000513,168.434276,-0.000509,2.072328e-04,...,0.000310,-4.223756e-07,1.891914e-08,16384.0,0.000011,1.246574e-10,0.000011,0.000063,0.000063,-0.000050
A1_NDEC,-0.000029,0.0,0.000029,180.0,-0.000029,-2.677937e-05,0.000040,-137.414962,-0.000028,-5.401134e-05,...,-0.000077,-7.826342e-08,-4.716090e-09,16384.0,0.000009,7.658542e-11,0.000009,0.000030,0.000035,-0.000035
A3_EC,-0.000122,0.0,0.000122,180.0,-0.000121,3.280312e-05,0.000126,164.856286,-0.000119,6.683558e-05,...,-0.000472,2.496537e-07,-2.882449e-08,16384.0,0.000010,1.085926e-10,0.000010,0.000033,0.000046,-0.000046
A3_EO,0.000066,0.0,0.000066,0.0,0.000067,-2.507370e-05,0.000072,-20.471719,0.000069,-5.023613e-05,...,0.000192,-7.100053e-08,1.170106e-08,16384.0,0.000007,4.976160e-11,0.000007,0.000022,0.000030,-0.000030
A3_DEC,-0.000053,0.0,0.000053,180.0,-0.000052,-3.083023e-05,0.000061,-149.366337,-0.000050,-6.271560e-05,...,-0.000051,2.889488e-07,-3.108092e-09,16384.0,0.000007,4.420522e-11,0.000007,0.000025,0.000026,-0.000026
A3_NDEC,0.000373,0.0,0.000373,0.0,0.000375,-9.920712e-06,0.000375,-1.517104,0.000381,-1.978496e-05,...,0.000424,-2.654945e-07,2.585556e-08,16384.0,0.000007,4.282515e-11,0.000007,0.000021,0.000021,-0.000020
A4_EC,0.000040,0.0,0.000040,0.0,0.000039,-2.014617e-05,0.000044,-27.035305,0.000039,-4.032384e-05,...,0.000545,8.947322e-07,3.326021e-08,16384.0,0.000020,3.982424e-10,0.000020,0.000070,0.000070,-0.000060
A4_EO,-0.001495,0.0,0.001495,180.0,-0.001500,-9.948582e-05,0.001503,-176.205330,-0.001515,-2.005598e-04,...,-0.001381,-1.524369e-08,-8.428373e-08,16384.0,0.000007,5.431797e-11,0.000007,0.000039,0.000065,-0.000065


In [82]:
import pickle

# Assuming `best_clf` is the latest trained XGBoost model
with open('88%_xgboost_model.pkl', 'wb') as file:
    pickle.dump(best_clf, file)

print("XGBoost model saved successfully.")

XGBoost model saved successfully.
